In [1]:
import pandas as pd
import requests
import json
from datetime import datetime
from pathlib import Path
import os
import time

In [2]:
key = 'AU74VSFGT1S37O4A'
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [3]:
#This gets the parent directory of the current directory
current = os.getcwd()
print(current)

C:\Users\Russell\OneDrive - Marist College\School\Spring 2020\DATA 450 Data Capping\Repository\VM


In [4]:
def format_daily_api(stockResponse):
    newDf = pd.DataFrame(columns =["open", "high", "low", "close", 'volume','time'])
    
    for times in stockResponse['Time Series (Daily)'].keys():
        timestamp = datetime.strptime(times, '%Y-%m-%d')
        ope = stockResponse['Time Series (Daily)'][times]['1. open']
        high = stockResponse['Time Series (Daily)'][times]['2. high']
        low = stockResponse['Time Series (Daily)'][times]['3. low']
        close = stockResponse['Time Series (Daily)'][times]['4. close']
        volume = stockResponse['Time Series (Daily)'][times]['5. volume']
        
        newDf = newDf.append({"open":ope, "high":high, "low":low,
                          "close":close, "volume":volume, "time":timestamp}, ignore_index=True)
    newDf['time'] = pd.to_datetime(newDf['time'])
    
    return newDf

In [5]:
for symbol in stockSymbol:
    dailyUrl = ('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&apikey='+key).format(symbol)
    stockRes = requests.get(dailyUrl).json()
    
    companyFolder = str(current)+"\\{}folder".format(symbol)
    
    file = companyFolder+'\\{}Daily.csv'.format(symbol)
    df=pd.read_csv(file)
    df.time = pd.to_datetime(df.time)
    
    stockAPIdf = format_daily_api(stockRes)

    ##Stack the dataframes on top of each other without the issue of different columns
    newDailyValueDf = stockAPIdf.append(df)
    
    
    newDailyValueDf['percentChange'] = newDailyValueDf['close'].astype(float).pct_change(periods=-1)
    newDailyValueDf['percentVol'] = newDailyValueDf['volume'].astype(float).pct_change(periods=-1)
    
    dataframeValue = newDailyValueDf[{'open', 'high', 'low', 'close', 'volume', 'time', 'percentChange', 'percentVol'}]
    
    dataframeValue['time'] = pd.to_datetime(dataframeValue['time'])
    
    dataframeValue = dataframeValue.sort_values(by= 'time', ascending = False)
    
    dataframeValue = dataframeValue.drop_duplicates(subset='time')
    
    dataframeValue = dataframeValue[{'open', 'high', 'low', 'close', 'volume', 'time', 'percentChange', 'percentVol'}]
    
    dataframeValue.to_csv(file, index=False)
    time.sleep(30)
    time.sleep(30)
    
for stock in stockSymbol:

    newDf = pd.DataFrame(columns =["open", "high", "low", "close", 'volume','time'])


    url = ('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=1min&outputsize=full&apikey='+key).format(stock)
    stockRes = requests.get(url).json()
    
    companyFolder = str(current)+"\\{}folder".format(stock)

    file = companyFolder+'\\{}Values.csv'.format(stock)
    df=pd.read_csv(file)
    df.time = pd.to_datetime(df.time)

    for times in stockRes['Time Series (1min)'].keys():
        timestamp = datetime.strptime(times, '%Y-%m-%d %H:%M:%S')
        ope = stockRes['Time Series (1min)'][times]['1. open']
        high = stockRes['Time Series (1min)'][times]['2. high']
        low = stockRes['Time Series (1min)'][times]['3. low']
        close = stockRes['Time Series (1min)'][times]['4. close']
        volume = stockRes['Time Series (1min)'][times]['5. volume']

        newDf = newDf.append({"open":ope, "high":high, "low":low, "close":close, "volume":volume, "time":timestamp}, ignore_index=True)

    dataframeValue = newDf.append(df)

    dataframeValue = dataframeValue[{'open', 'high', 'low', 'close', 'volume', 'time'}]

    dataframeValue['time'] = pd.to_datetime(dataframeValue['time'])

    dataframeValue = dataframeValue.drop_duplicates(subset='time')

    dataframeValue = dataframeValue.sort_values(by = 'time', ascending = False)

    dataframeValue.to_csv(file, index=False)
    time.sleep(60)